In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('data/train.csv')

In [3]:
test_df = pd.read_csv('data/test.csv')

## Feature Engineering

In [4]:
print("The lowest date in the train data set is {} and the highest {}.".format(train_df['Date'].min(),train_df['Date'].max()))
print("The lowest date in the test data set is {} and the highest {}.".format(test_df['Date'].min(),test_df['Date'].max()))

The lowest date in the train data set is 2020-01-23 and the highest 2020-06-02.
The lowest date in the test data set is 2020-04-27 and the highest 2020-06-10.


In [5]:
train_df.rename(columns={'Province_State':'State','Country_Region':'Country'}, inplace=True)
test_df.rename(columns={'Province_State':'State','Country_Region':'Country'}, inplace=True)

In [6]:
#function for replacing all the missings in the state column
def missings(state, country):
    return country if pd.isna(state) == True else state

In [7]:
#if there are no states specified for a country, the missing is replaced with the country´s name
train_df['State'] = train_df.apply(lambda x: missings(x['State'],x['Country']),axis=1)
test_df['State'] = test_df.apply(lambda x: missings(x['State'],x['Country']),axis=1)

In [8]:
print("In our data set are {} countries and {} states.".format(train_df['Country'].nunique(),train_df['State'].nunique()))

In our data set are 187 countries and 319 states.


In [9]:
train_df.head()

,Id,County,State,Country,Population,Weight,Date,Target,TargetValue
0,1,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-01-23,ConfirmedCases,0.0
1,2,NaN,Afghanistan,Afghanistan,27657145,0.583587,2020-01-23,Fatalities,0.0
2,3,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-01-24,ConfirmedCases,0.0
3,4,NaN,Afghanistan,Afghanistan,27657145,0.583587,2020-01-24,Fatalities,0.0
4,5,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-01-25,ConfirmedCases,0.0


In [10]:
train_df['Date'] = pd.to_datetime(train_df['Date'])

In [11]:
mask = (train_df['Date'] > '2020-05-01')

In [12]:
df = train_df.loc[mask]

In [13]:
df.head()

,Id,County,State,Country,Population,Weight,Date,Target,TargetValue
200,201,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-02,ConfirmedCases,134.0
201,202,NaN,Afghanistan,Afghanistan,27657145,0.583587,2020-05-02,Fatalities,4.0
202,203,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-03,ConfirmedCases,235.0
203,204,NaN,Afghanistan,Afghanistan,27657145,0.583587,2020-05-03,Fatalities,13.0
204,205,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-04,ConfirmedCases,190.0


In [14]:
confirmed_cases_df = df[df['Target']=='ConfirmedCases'][['Id', 'County', 'State', 'Country', 'Population','Weight', 'Date','TargetValue']]
fatalities_df = df[df['Target']=='Fatalities'][['Id', 'County', 'State', 'Country', 'Population','Weight', 'Date','TargetValue']]

In [15]:
confirmed_cases_df.head()

,Id,County,State,Country,Population,Weight,Date,TargetValue
200,201,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-02,134.0
202,203,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-03,235.0
204,205,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-04,190.0
206,207,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-05,330.0
208,209,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-06,168.0


In [16]:
fatalities_df.head()

,Id,County,State,Country,Population,Weight,Date,TargetValue
201,202,NaN,Afghanistan,Afghanistan,27657145,0.583587,2020-05-02,4.0
203,204,NaN,Afghanistan,Afghanistan,27657145,0.583587,2020-05-03,13.0
205,206,NaN,Afghanistan,Afghanistan,27657145,0.583587,2020-05-04,5.0
207,208,NaN,Afghanistan,Afghanistan,27657145,0.583587,2020-05-05,5.0
209,210,NaN,Afghanistan,Afghanistan,27657145,0.583587,2020-05-06,9.0


In [17]:
confirmed_cases_df.to_csv('data/confirmed_cases.csv')

In [18]:
fatalities_df.to_csv('data/fatalities.csv')

In [19]:
result = pd.merge(confirmed_cases_df, fatalities_df, on=['Date', 'Country'])

In [20]:
result.head()

,Id_x,County_x,State_x,Country,Population_x,Weight_x,Date,TargetValue_x,Id_y,County_y,State_y,Population_y,Weight_y,TargetValue_y
0,201,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-02,134.0,202,NaN,Afghanistan,27657145,0.583587,4.0
1,203,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-03,235.0,204,NaN,Afghanistan,27657145,0.583587,13.0
2,205,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-04,190.0,206,NaN,Afghanistan,27657145,0.583587,5.0
3,207,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-05,330.0,208,NaN,Afghanistan,27657145,0.583587,5.0
4,209,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-06,168.0,210,NaN,Afghanistan,27657145,0.583587,9.0


In [23]:
intermediate_data = result[['Id_x', 'County_x', 'State_x', 'Country', 
                  'Population_x','Weight_x', 'Date','TargetValue_x', 'TargetValue_y']]

In [25]:
import gc
del result
del train_df
del confirmed_cases_df
del fatalities_df
gc.collect()
intermediate_data.rename(columns={'TargetValue_x':'ConfirmedCases', 
                       'TargetValue_y':'Fatalities',
                       'Id_x': 'Id', 
                       'County_x': 'County', 
                       'State_x': 'State', 
                       'Population_x': 'Population',
                       'Weight_x': 'Weight',}, inplace=True)

In [26]:
intermediate_data.head()

,Id,County,State,Country,Population,Weight,Date,ConfirmedCases,Fatalities
0,201,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-02,134.0,4.0
1,203,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-03,235.0,13.0
2,205,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-04,190.0,5.0
3,207,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-05,330.0,5.0
4,209,NaN,Afghanistan,Afghanistan,27657145,0.058359,2020-05-06,168.0,9.0


In [27]:
# Confirmed cases from June 1st
df_confirmedcases = intermediate_data.groupby(['Country','State']).max().groupby('Country').sum().sort_values(by='ConfirmedCases', ascending=False).reset_index().drop(columns='Id')
df_confirmedcases[:20].set_index('Country').style.background_gradient(cmap='Oranges')

,Population,Weight,ConfirmedCases,Fatalities
Country,,,,
US,648286272,6.765592,78422.000000,130185.000000
Brazil,206135893,0.052236,33274.000000,1262.000000
Russia,146599183,0.053182,11656.000000,232.000000
India,1295210000,0.047660,8821.000000,269.000000
Peru,31488700,0.057920,8805.000000,195.000000
United Kingdom,65600525,1.048416,6152.000000,7623.000000
Canada,75700840,0.947336,5516.000000,2795.000000
Chile,18191900,0.059821,5470.000000,75.000000
Pakistan,194125062,0.052400,3938.000000,88.000000


In [29]:
print("The lowest date in the train data set is {} and the highest {}.".format(intermediate_data['Date'].min(),intermediate_data['Date'].max()))

The lowest date in the train data set is 2020-05-02 00:00:00 and the highest 2020-06-02 00:00:00.


In [31]:
type(intermediate_data)

pandas.core.frame.DataFrame